# MQTT-SN between FIT and Grid'5000

Prerequisites:
- Be connected to this Jupyter lab instance https://labs.iot-lab.info
## One time Setup

First things first, you'll need EnOSlib library to go through this tutorial.
- Launch the following
- Restart the kernel

In [1]:
!pip install -U --pre --user --no-cache enoslib

In [2]:
# This must not fail (restart the kernel after installing the library)
import enoslib as en

The Vagrant executable cannot be found. Please check if it is in the system path.
Using custom api_url: https://www.iot-lab.info/api/


Note: Openstack clients not installed


### Setup access to Grid'5000 

We need two kind of accesses to the Grid'5000 platform:
- REST API Access to interact with  Grid'5000 exposed resources (jobs, nodes, networks, firewall ....)
- SSH Access to control the remote machine once acquired

---
The REST API access is performed using HTTP request using BasicAuth Authentication mecanism through the [`python-grid5000`](https://pypi.org/project/python-grid5000/) wrapper.
This requires to store the username/password in a file located in your home directory.

In [3]:
from grid5000.cli import auth

# will prompt for the password and write the authentication file
auth("msimonin")

Grid'5000 password:  ········


/home/jovyan/.python-grid5000.yaml created successfully


---
The SSH access relies on SSH key authentication.
Add the content of the public key in the Grid'5000 interface in [your grid'5000 account](https://api.grid5000.fr/stable/users/) > ssh_keys

In [4]:
!cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQC1GkxHp8fSTVcQfOSc3dDndPYgU906XcF4DlWkX7pP8KEYoaDhSkIrtwYNCyUylj9uPzze8rB8qFaSDHlorb/sz5YfAhaHfvwqSDQXeHa0O14dtQOWzYj8sSznA3JSrHZCvSzG6v8QhvJZ6Bt4SZp7R0e2pIOwqDMcNwy+V2RNt8puvDkJyqbYk7ecPnRBSNL9EGxfLWB2hRLW8nAKuVWKLAPEVqmC4Mp9Ij5cNMMugMReWK9CvnDxvwB5ZgM55NHU4a6ilsMAIpZidm04YxKA/X4Cb5VtOIpHy0YtxFmEvwrLX++gUQ7U8AFXUz2/8CYWtJRjXVZWP7pbfZN55lXznXMwfvpD3YrxdTMrYS1HPMCq6PandZsmB8u3htBmPum5GsaIYkZOW9g9kLQfsUqZSWpE4r3opM/fWOD8iXXdM2QHdIK1Ix31Bg3F8ir1wcXj0W67D6qzBIlU0wlhDuSJnrZ8QTTvCri0iG+iEvJlYXZYG+D0KE+szW4L1FSOwe0= jupyterhub@iotlab


## FIT side

### Firmware compilations

In [5]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID=0x{}'.format(channel, pan_id))

Use CHANNEL=17, PAN_ID=0x157e


In [6]:
%env CHANNEL=14
%env PAN_ID=0xce70
%env ETHOS_BAUDRATE=500000

env: CHANNEL=14
env: PAN_ID=0xce70
env: ETHOS_BAUDRATE=500000


In [7]:
%env APP_DIR=../../../../riot/RIOT/examples/gnrc_border_router
!make -C $APP_DIR BOARD=iotlab-m3 DEFAULT_CHANNEL=$CHANNEL DEFAULT_PAN_ID=$PAN_ID && cp $APP_DIR/bin/iotlab-m3/*.bin .

env: APP_DIR=../../../../riot/RIOT/examples/gnrc_border_router
make: Entering directory '/home/jovyan/work/training/riot/RIOT/examples/gnrc_border_router'
Building application "gnrc_border_router" for "iotlab-m3" with MCU "stm32".

"make" -C /home/jovyan/work/training/riot/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/training/riot/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/training/riot/RIOT/core
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/periph
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/stmclk
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/vectors
"make" -C /home/jovyan/work/training/riot/RIOT/drivers
"make" -C /home/jovyan/work/training/riot/RIOT/drivers/at86rf2xx
"make" -C /home/jovyan/work/training/riot/RIOT/drivers/ethos
"make" -C /hom

In [8]:
%env APP_DIR=../../../../riot/RIOT/examples/emcute_mqttsn
!make -C $APP_DIR BOARD=iotlab-m3 DEFAULT_CHANNEL=$CHANNEL DEFAULT_PAN_ID=$PAN_ID && cp $APP_DIR/bin/iotlab-m3/*.bin .

env: APP_DIR=../../../../riot/RIOT/examples/emcute_mqttsn
make: Entering directory '/home/jovyan/work/training/riot/RIOT/examples/emcute_mqttsn'
Building application "emcute_mqttsn" for "iotlab-m3" with MCU "stm32".

"make" -C /home/jovyan/work/training/riot/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/training/riot/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/training/riot/RIOT/core
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/periph
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/stmclk
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32/vectors
"make" -C /home/jovyan/work/training/riot/RIOT/drivers
"make" -C /home/jovyan/work/training/riot/RIOT/drivers/at86rf2xx
"make" -C /home/jovyan/work/training/riot/RIOT/drivers/netdev
"make" -C /home/jovyan/work/

### Get some resources

In [1]:
import enoslib as en

en.init_logging()
FIT_SITE = "grenoble"

The Vagrant executable cannot be found. Please check if it is in the system path.
Using custom api_url: https://www.iot-lab.info/api/


Note: Openstack clients not installed


In [2]:
fit_conf = (
    en.IotlabConf.from_settings(job_name="riot_m3", walltime="02:00")
    .add_machine(roles=["border_router"], archi="m3:at86rf231", site=FIT_SITE, number=1, image="gnrc_border_router.bin")    
    .add_machine(roles=["other"], archi="m3:at86rf231", site=FIT_SITE, number=1, image="emcute_mqttsn.bin")    
)
fit_conf

roles,image,archi,site,number
border_router,gnrc_border_router.bin,m3:at86rf231,grenoble,1
other,emcute_mqttsn.bin,m3:at86rf231,grenoble,1


In [3]:
fit = en.Iotlab(fit_conf)
fit_roles, _ = fit.init()

INFO     Waiting for job id (303860) to be in running state                 ]8;id=472688;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=752518;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#174\174]8;;\

INFO     Job id (303860) is running                                         ]8;id=871560;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=218092;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#176\176]8;;\

INFO     Finished reserving nodes: hosts [], sensors                          ]8;id=629019;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/provider.py\provider.py]8;;\:]8;id=42794;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/provider.py#213\213]8;;\
         [<IotlabSensor(roles=['border_router'],                                             
         address=m3-100.grenoble.iot-lab.info, site=grenoble,                                
         uid=b080)>image=gnrc_border_router.bin)>,                                           
         <IotlabSensor(roles=['other'], address=m3-101.grenoble.iot-lab.info,                
         site=grenoble, uid=9181)>image=emcute_mqttsn.bin)>]                                 

INFO     Flashing image (gnrc_border_router.bin) on nodes                   ]8;id=477268;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=805751;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#219\219]8;;\
         (['m3-100.grenoble.iot-lab.info'])                                                  

INFO     Flashing image (emcute_mqttsn.bin) on nodes                        ]8;id=119724;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=435134;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#219\219]8;;\
         (['m3-101.grenoble.iot-lab.info'])                                                  

In [4]:
br = fit_roles["border_router"][0]
br.reset()

INFO     Executing command (reset) on nodes                                 ]8;id=498269;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=324200;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#274\274]8;;\
         (['m3-100.grenoble.iot-lab.info'])                                                  

In [5]:
other = fit_roles["other"][0]
other.reset()

INFO     Executing command (reset) on nodes                                 ]8;id=122336;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py\iotlab_api.py]8;;\:]8;id=404672;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_iotlab/iotlab_api.py#274\274]8;;\
         (['m3-101.grenoble.iot-lab.info'])                                                  

### Setting up IPv6

In [6]:
import iotlabcli
iotlab_user, _ = iotlabcli.auth.get_user_credentials()
fit_frontend = en.Host("%s.iot-lab.info" % FIT_SITE, alias=FIT_SITE, user=iotlab_user) 

At this point you need to pick a free tap number and a [free subnet](https://www.iot-lab.info/legacy/tutorials/understand-ipv6-subnetting-on-the-fit-iot-lab-testbed/index.html)  

In [7]:
r = en.run(f"ip -6 route", roles=fit_frontend)
print(r[0].stdout)

Output()

Finished 1 tasks (ip -6 route)

─────────────────────────────────────────────────────────────────────────────────────────────

::1 dev lo proto kernel metric 256 pref medium
2001:660:5307:30ff::/64 dev ens3 proto kernel metric 256 pref medium
fe80::/64 dev ens3 proto kernel metric 256 pref medium
fe80::/64 dev ens7 proto kernel metric 256 pref medium
default via 2001:660:5307:30ff:ff:: dev ens3 metric 1024 onlink pref medium


In [8]:
# change me
tap = "tap43"
prefix = "2001:660:5307:3144"

In [9]:
en.run(f"ps aux | grep tap43 | grep {iotlab_user}| grep -v grep | awk '{{print $2}}' | xargs -r kill", roles=fit_frontend)
r = en.run(f"sudo ethos_uhcpd.py {br.alias} {tap} {prefix}::1/64 2>&1 >> ethos_uhcpd.out", roles=fit_frontend, background=True)

Output()

Finished 1 tasks (ps aux | grep tap43 | grep simonin| grep -v grep | awk '{print $2}' | xargs
-r kill)

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (sudo ethos_uhcpd.py m3-100 tap43 2001:660:5307:3144::1/64 2>&1 >> 
ethos_uhcpd.out)

─────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
r = en.run(f"ip -6 route", roles=fit_frontend)
print(r[0].stdout)

Output()

Finished 1 tasks (ip -6 route)

─────────────────────────────────────────────────────────────────────────────────────────────

::1 dev lo proto kernel metric 256 pref medium
2001:660:5307:30ff::/64 dev ens3 proto kernel metric 256 pref medium
2001:660:5307:3144::/64 via fe80::2 dev tap43 metric 1024 pref medium
fe80::/64 dev ens3 proto kernel metric 256 pref medium
fe80::/64 dev ens7 proto kernel metric 256 pref medium
fe80::/64 dev tap43 proto kernel metric 256 pref medium
default via 2001:660:5307:30ff:ff:: dev ens3 metric 1024 onlink pref medium


The other M3 device should get its IPv6 in the chosen after few seconds.
Retry the following until you get this IPv6 (or until you get bored)

Note that the command sends a command (`ifconfig`) to the other M3 device after SSH-ing to the frontend.

In [11]:
r = en.run(f"echo ifconfig | nc -w1  {other.alias} 20000", fit_frontend)
print(r[0].stdout)

Output()

Finished 1 tasks (echo ifconfig | nc -w1  m3-101 20000)

─────────────────────────────────────────────────────────────────────────────────────────────

ifconfig
Iface  5  HWaddr: 1D:60  Channel: 14  Page: 0  NID: 0xce70  PHY: O-QPSK 
          
          Long HWaddr: A6:DA:E9:93:B3:CC:9D:60 
           TX-Power: 0dBm  State: IDLE  max. Retrans.: 3  CSMA Retries: 4 
          AUTOACK  ACK_REQ  CSMA  L2-PDU:102  MTU:1280  HL:64  6LO  
          IPHC  
          Source address length: 8
          Link type: wireless
          inet6 addr: fe80::a4da:e993:b3cc:9d60  scope: link  VAL
          inet6 addr: 2001:660:5307:3144:a4da:e993:b3cc:9d60  scope: global  VAL
          inet6 group: ff02::1
          
> 


In [12]:
# record this for future use
FIT_IPv6 = "2001:660:5307:3144:a4da:e993:b3cc:9d60"

In [13]:
# connectivity test with ... google
r = en.run(f"echo ping6 2001:4860:4860::8888 | nc -w5  {other.alias} 20000", fit_frontend)
print(r[0].stdout)

Output()

Finished 1 tasks (echo ping6 2001:4860:4860::8888 | nc -w5  m3-101 20000)

─────────────────────────────────────────────────────────────────────────────────────────────

ping6 2001:4860:4860::8888
12 bytes from 2001:4860:4860::8888: icmp_seq=0 ttl=113 rssi=-55 dBm time=51.849 ms
12 bytes from 2001:4860:4860::8888: icmp_seq=1 ttl=113 rssi=-55 dBm time=34.339 ms
12 bytes from 2001:4860:4860::8888: icmp_seq=2 ttl=113 rssi=-55 dBm time=27.130 ms

--- 2001:4860:4860::8888 PING statistics ---
3 packets transmitted, 3 packets received, 0% packet loss
round-trip min/avg/max = 27.130/37.772/51.849 ms
> 


## Grid'5000 side

### Get some resources

In [14]:
network = en.G5kNetworkConf(type="prod", roles=["my_network"], site="rennes")

conf = (
    en.G5kConf.from_settings(job_type="allow_classic_ssh", job_name="rsd-01")
    .add_network_conf(network)
    .add_machine(
        roles=["broker"], cluster="paravance", nodes=1, primary_network=network
    )
    .finalize()
)
conf

Conf@0x7f5d3c41f0a0
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian11-x64-nfs",
    "job_name": "rsd-01",
    "job_type": "allow_classic_ssh",
    "key": "/home/jovyan/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "broker"
                ],
                "primary_network": "57df626b-3a22-4788-9267-2857d1084be6",
                "secondary_networks": [],
                "cluster": "paravance",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "57df626b-3a22-4788-9267-2857d1084be6",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}

In [15]:
g5k = en.G5k(conf)
g5k_roles, g5k_networks = g5k.init()

INFO     Reloading rsd-01 from grenoble                                  ]8;id=746071;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=447950;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from lille                                     ]8;id=246675;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=909864;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from luxembourg                                ]8;id=186571;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=109316;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from lyon                                      ]8;id=226717;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=687531;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from nancy                                     ]8;id=438267;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=570341;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

...

INFO     Reloading rsd-01 from rennes                                    ]8;id=324778;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=200047;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from sophia                                    ]8;id=135640;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=838487;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Submitting {'name': 'rsd-01', 'types': ['allow_classic_ssh'],   ]8;id=138338;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=27339;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#286\286]8;;\
         'resources': "{cluster='paravance'}/nodes=1,walltime=02:00:00",                     
         'command': 'sleep 31536000', 'queue': 'default'} on rennes                          

INFO     Waiting for 1879409 on rennes [2022-03-09 16:53:21]             ]8;id=784451;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=961998;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#317\317]8;;\

INFO     All jobs are Running !                                          ]8;id=78771;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=642838;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#324\324]8;;\

### Setting up IPv6

In [16]:
with en.actions(roles=g5k_roles) as a:
    # enable ipv6
    a.shell("dhclient -6 br0")
    
r = en.run("ip a", g5k_roles)
print(r[0].stdout)

Output()

Finished 1 tasks (shell)

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (ip a)

─────────────────────────────────────────────────────────────────────────────────────────────

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eno3: <BROADCAST,MULTICAST> mtu 1500 qdisc mq state DOWN group default qlen 1000
    link/ether ec:f4:bb:d0:ff:e4 brd ff:ff:ff:ff:ff:ff
    altname enp7s0f0
3: eno4: <BROADCAST,MULTICAST> mtu 1500 qdisc mq state DOWN group default qlen 1000
    link/ether ec:f4:bb:d0:ff:e5 brd ff:ff:ff:ff:ff:ff
    altname enp7s0f1
4: eno1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq master br0 state UP group default qlen 1000
    link/ether ec:f4:bb:d0:ff:e0 brd ff:ff:ff:ff:ff:ff
    altname enp1s0f0
5: eno2: <BROADCAST,MULTICAST> mtu 1500 qdisc mq state DOWN group default qlen 1000
    link/ether ec:f4:bb:d0:ff:e2 brd ff:ff:ff:ff:ff:ff
    altname enp1s0f1
6: br0: <BROADCAST,MULTICAST,U

In [17]:
G5K_IPv6 = "2001:660:4406:700:1::47"

## Cross traffic connectivity 

### Grid'5000 -> FIT

In [18]:
r = en.run(f"ping -c 5 {FIT_IPv6}", roles=g5k_roles)
print(r[0].stdout)

Output()

Finished 1 tasks (ping -c 5 2001:660:5307:3144:a4da:e993:b3cc:9d60)

─────────────────────────────────────────────────────────────────────────────────────────────

PING 2001:660:5307:3144:a4da:e993:b3cc:9d60(2001:660:5307:3144:a4da:e993:b3cc:9d60) 56 data bytes
64 bytes from 2001:660:5307:3144:a4da:e993:b3cc:9d60: icmp_seq=2 ttl=51 time=104 ms
64 bytes from 2001:660:5307:3144:a4da:e993:b3cc:9d60: icmp_seq=3 ttl=51 time=62.7 ms
64 bytes from 2001:660:5307:3144:a4da:e993:b3cc:9d60: icmp_seq=4 ttl=51 time=53.1 ms
64 bytes from 2001:660:5307:3144:a4da:e993:b3cc:9d60: icmp_seq=5 ttl=51 time=59.8 ms

--- 2001:660:5307:3144:a4da:e993:b3cc:9d60 ping statistics ---
5 packets transmitted, 4 received, 20% packet loss, time 4032ms
rtt min/avg/max/mdev = 53.051/69.912/104.138/20.065 ms


### FIT -> Grid5000

In [ ]:
# open the firewall for the duration of the inner command
with g5k.firewall(proto="all"):
    r = en.run(f"echo ping6 {G5K_IPv6} | nc -w5  {other.alias} 20000", fit_frontend)
    print(r[0].stdout)

INFO     Reloading rsd-01 from grenoble                                  ]8;id=343444;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=36721;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from lille                                     ]8;id=740264;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=148591;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from luxembourg                                ]8;id=719302;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=708642;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from lyon                                      ]8;id=436995;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=380659;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from nancy                                     ]8;id=235909;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=347310;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from nantes                                    ]8;id=503785;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=295709;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading rsd-01 from rennes                                    ]8;id=388857;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=653151;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

INFO     Reloading 1879409 from rennes                                   ]8;id=677491;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=679249;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#180\180]8;;\

INFO     Reloading rsd-01 from sophia                                    ]8;id=436901;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=911662;file:///home/jovyan/.local/lib/python3.9/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#175\175]8;;\

Output()

## MQTT FIT <-> G5K

In [ ]:
# TODO